In [4]:
import csv
import numpy as np

file_path = 'files/feature_table.csv'
# Lendo o arquivo CSV e armazenando os dados em uma lista de listas
data = []
with open(file_path, newline='') as csvfile:
    csv_reader = csv.reader(csvfile)
    for row in csv_reader:
        del row[0]
        data.append(row)



# Removendo a primeira linha
del data[0]
del data[0]
del data[100:]
#for i in range(len(data)):
#   del data[i][0]

for i in range(len(data)):
    for j in range(len(data[i])):
        if float(data[i][j]) > 0:
            data[i][j] = 1
        else:
            data[i][j] = 0

matrix_size = len(data) #dimensão da matriz de adjacência

#matriz_original = [[0, 0, 1], [1, 0, 1], [1, 0, 0], [1, 1, 1], [1, 1, 0]] #matriz com 5 linhas e 3 colunas
matriz_adj = np.zeros((matrix_size, matrix_size)) #criando a matriz de adjacência com zeros

for i in range(len(matriz_adj)):
  for j in range(len(matriz_adj)):
    if i != j:
      for k in range(len(data[i])):
        if data[i][k] == 1 and data[j][k] == 1:
          matriz_adj[i][j] = 1
          break
for row in matriz_adj:
   print(row)

N = {
    i: set(num for num, j in enumerate(row) if j)
    for i, row in enumerate(matriz_adj)
}

#print(N)
# {0: {1, 4}, 1: {0, 2, 4}, 2: {1, 3}, 3: {2, 4, 5}, 4: {0, 1, 3}, 5: {3}}

def BronKerbosch1(P, R=None, X=None):
    P = set(P) # 1
    R = set() if R is None else R # 1
    X = set() if X is None else X # 1
    if not P and not X: # 1
        yield R # 1
    while P:
        v = P.pop() # 1
        yield from BronKerbosch1(
            P=P.intersection(N[v]), R=R.union([v]), X=X.intersection(N[v])) # 3^n
        X.add(v)

P = N.keys()
#print(P)
print(list(BronKerbosch1(P)))


[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1.